In [1]:
import sys
!{sys.executable} -m pip install --upgrade --user pip # update the pip package installer
!{sys.executable} -m pip install -U numpy pandas uproot3 matplotlib --user # install required packages

  Using cached pip-21.3.1-py3-none-any.whl (1.7 MB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Using cached numpy-1.21.4-cp39-cp39-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Using cached pandas-1.3.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.5 MB)
  Using cached uproot3-3.14.4-py3-none-any.whl (117 kB)
  Using cached matplotlib-3.5.0-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
  Using cached setuptools_scm-6.3.2-py3-none-any.whl (33 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.5.0 requires keras-nightly~=2.5.0.dev, which is not installed.
tensorboard 2.5.0 requires tensorboard-data-server<0.7.0,>=0.6.

In [2]:
import pandas as pd # to store data as dataframes
import numpy as np # for numerical calculations such as histogramming
import math # for mathematical functions such as square root
import uproot3 # to read .root files as dataframes
import matplotlib.pyplot as plt # for plotting
from matplotlib.ticker import AutoMinorLocator # for minor ticks
import infofile # local file containing info on cross-sections, sums of weights, dataset IDs

/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc10-opt/lib/python3.9/site-packages/uproot3/__init__.py:127: FutureWarning: Consider switching from 'uproot3' to 'uproot', since the new interface became the default in 2020.

    pip install -U uproot

In Python:

    >>> import uproot
    >>> with uproot.open(...) as file:
    ...

  warnings.warn(
/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc10-opt/lib/python3.9/site-packages/awkward0/__init__.py:12: FutureWarning: Consider switching from 'awkward0' to 'awkward', since the new interface became the default in 2020.

    pip install -U awkward

In Python:

    >>> import awkward as ak
    >>> new_style_array = ak.from_awkward0(old_style_array)
    >>> old_style_array = ak.to_awkward0(new_style_array)

  warnings.warn(


In [3]:
#lumi = 0.5 # fb-1 # data_A only
#lumi = 1.9 # fb-1 # data_B only
#lumi = 2.9 # fb-1 # data_C only
#lumi = 4.7 # fb-1 # data_D only
lumi = 10 # fb-1 # data_A,data_B,data_C,data_D

fraction = 1.0 # reduce this is if you want the code to run quicker
                                                                                                                                  
#tuple_path = "Input/4lep/" # local 
tuple_path = "https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/4lep/" # web address

In [4]:
samples = {

    'data': {
        'list' : ['data_A','data_B','data_C','data_D'],
    },

    r'Background $Z,t\bar{t}$' : { # Z + ttbar
        'list' : ['Zee','Zmumu','ttbar_lep'],
        'color' : "#6b59d3" # purple
    },

    r'Background $ZZ^*$' : { # ZZ
        'list' : ['llll'],
        'color' : "#ff0000" # red
    },
    
    
    r'Signal ($m_H$ = 125 GeV)' : { # H -> ZZ -> llll
        'list' : ['ggH125_ZZ4lep','VBFH125_ZZ4lep','WH125_ZZ4lep','ZH125_ZZ4lep'],
        'color' : "#00cdff" # light blue
    },
}

In [5]:
def get_data_from_files():

    data = {} # define empty dictionary to hold dataframes
    for s in samples: # loop over samples
        print('Processing '+s+' samples') # print which sample
        frames = [] # define empty list to hold data
        for val in samples[s]['list']: # loop over each file
            if s == 'data': prefix = "Data/" # Data prefix
            else: # MC prefix
                prefix = "MC/mc_"+str(infofile.infos[val]["DSID"])+"."
            fileString = tuple_path+prefix+val+".4lep.root" # file name to open
            temp = read_file(fileString,val) # call the function read_file defined below
            frames.append(temp) # append dataframe returned from read_file to list of dataframes
            #print(temp['lep_pt'])
            #print(temp['mllll'])
        data[s] = pd.concat(frames) # dictionary entry is concatenated dataframes
    
    return data # return dictionary of dataframes

In [6]:
def calc_weight(xsec_weight, mcWeight, scaleFactor_PILEUP,
                scaleFactor_ELE, scaleFactor_MUON, 
                scaleFactor_LepTRIGGER ):
    return (xsec_weight * mcWeight * scaleFactor_PILEUP * scaleFactor_ELE * scaleFactor_MUON  * scaleFactor_LepTRIGGER)

In [7]:
def get_xsec_weight(sample):
    info = infofile.infos[sample] # open infofile
    xsec_weight = (lumi*1000*info["xsec"])/(info["sumw"]*info["red_eff"]) #*1000 to go from fb-1 to pb-1
    return xsec_weight # return cross-section weight

In [8]:
def calc_mllll(lep_pt, lep_eta, lep_phi, lep_E):
    # first lepton is [0], 2nd lepton is [1] etc
    px_0 = lep_pt[0]*math.cos(lep_phi[0]) # x-component of lepton[0] momentum
    py_0 = lep_pt[0]*math.sin(lep_phi[0]) # y-component of lepton[0] momentum
    pz_0 = lep_pt[0]*math.sinh(lep_eta[0]) # z-component of lepton[0] momentum
    px_1 = lep_pt[1]*math.cos(lep_phi[1]) # x-component of lepton[1] momentum
    py_1 = lep_pt[1]*math.sin(lep_phi[1]) # y-component of lepton[1] momentum
    pz_1 = lep_pt[1]*math.sinh(lep_eta[1]) # z-component of lepton[1] momentum
    px_2 = lep_pt[2]*math.cos(lep_phi[2]) # x-component of lepton[2] momentum
    py_2 = lep_pt[2]*math.sin(lep_phi[2]) # y-component of lepton[2] momentum
    pz_2 = lep_pt[2]*math.sinh(lep_eta[2]) # z-component of lepton[2] momentum
    px_3 = lep_pt[3]*math.cos(lep_phi[3]) # x-component of lepton[3] momentum
    py_3 = lep_pt[3]*math.sin(lep_phi[3]) # y-component of lepton[3] momentum
    pz_3 = lep_pt[3]*math.sinh(lep_eta[3]) # z-component of lepton[3] momentum    
    sumpx = px_0 + px_1 + px_2 + px_3 # x-component of momentum
    sumpy = py_0 + py_1 + py_2 + py_3 # y-component of momentum
    sumpz = pz_0 + pz_1 + pz_2 + pz_3 # z-component of momentum 
    sump = math.sqrt(sumpx**2 + sumpy**2 + sumpz**2) # magnitude of momentum 
    sumE = lep_E[0] + lep_E[1] + lep_E[2] + lep_E[3] # energy of dilepton system
    return math.sqrt(sumE**2 - sump**2)/1000 #/1000 to go from MeV to GeV

In [9]:
def cuts (lep_charge, lep_type):
    cuts = False
    sum_lep_charge = lep_charge[0] + lep_charge[1] + lep_charge[2] + lep_charge[3]
    if sum_lep_charge != 0:
        sum_lep_type = lep_type[0] + lep_type[1] + lep_type[2] + lep_type[3]
        if ((sum_lep_type != 44) & (sum_lep_type != 48) & (sum_lep_type != 52)):
            cuts = True
    print(cuts==True)        
    return cuts

#def cut_lep_charge(lep_charge):     
    #print(lep_charge[0] + lep_charge[1] + lep_charge[2] + lep_charge[3] != 0)
    #return lep_charge[0] + lep_charge[1] + lep_charge[2] + lep_charge[3] != 0
    #cut1 = False
    # Cut #1: cut on lepton charge
    #sum_lep_charge = lep_charge[0] + lep_charge[1] + lep_charge[2] + lep_charge[3]
    #if sum_lep_charge != 0:
     #   cut1 = True
    #print(cut1==True)
    #return cut1==True 
    
#def cut_lep_type(lep_type):
    #sum_lep_type = lep_type[0] + lep_type[1] + lep_type[2] + lep_type[3]
    #return (sum_lep_type != 44) & (sum_lep_type != 48) & (sum_lep_type != 52)
    # Cut #2: cut on lepton type
    #cut2 = False
    #sum_lep_type = lep_type[0] + lep_type[1] + lep_type[2] + lep_type[3]    
    #if sum_lep_type != 44 & sum_lep_type != 48 & sum_lep_type != 52:
     #   cut2 = True
    #print(cut2==True)          
    #return cut2==True

In [10]:
def read_file(path,sample):
    
    print("\tProcessing: "+sample) # print which sample is being processed
    data_all = pd.DataFrame() # define empty pandas DataFrame to hold all data for this sample
    tree = uproot3.open(path)["mini"] # open the tree called mini
    numevents = uproot3.numentries(path, "mini") # number of events
    if 'data' not in sample: xsec_weight = get_xsec_weight(sample) # get cross-section weight
    for data in tree.iterate(['lep_charge','lep_type','lep_pt',
                              'lep_eta','lep_phi','lep_E', 
                              'mcWeight','scaleFactor_PILEUP',
                              'scaleFactor_ELE','scaleFactor_MUON',
                              'scaleFactor_LepTRIGGER'
                             ], # variables to calculate Monte Carlo weight
                             outputtype=pd.DataFrame, # choose output type as pandas DataFrame
                             entrystop=numevents*fraction): # process up to numevents*fraction

        nIn = len(data.index) # number of events in this batch
        print(data[['lep_pt', 'mcWeight']])
        if 'data' not in sample: # only do this for Monte Carlo simulation files
            # multiply all Monte Carlo weights and scale factors together to give total weight
        
            data['totalWeight'] = np.vectorize(calc_weight)(xsec_weight,
                                                            data.mcWeight,
                                                            data.scaleFactor_PILEUP,
                                                            data.scaleFactor_ELE,
                                                            data.scaleFactor_MUON,
                                                            data.scaleFactor_LepTRIGGER)

        
        # cut on lepton charge using the function cut_lep_charge defined above
        data = data[ np.vectorize(cut_lep_charge)(data.lep_charge)]
        #data = data[ np.vectorize(~cut_lep_charge)(data.lep_charge)]
        
        # cut on lepton type using the function cut_lep_type defined above
        data = data[ np.vectorize(cut_lep_type)(data.lep_type)]        
        #data = data[ np.vectorize(~cut_lep_type)(data.lep_type)]
        
        # calculation of 4-lepton invariant mass using the function calc_mllll defined above
        data['mllll'] = np.vectorize(calc_mllll)(data.lep_pt,data.lep_eta,data.lep_phi,data.lep_E)
        #data['lep_pt'])
        nOut = len(data.index) # number of events passing cuts in this batch
        data_all = data_all.append(data) # append dataframe from this batch to the dataframe for the whole sample
        print(data[['lep_pt', 'mllll']])
        print("\t\t nIn: "+str(nIn)+",\t nOut: \t"+str(nOut)) # events before and after
    data_all
    return data_all # return dataframe containing events passing all cuts


In [11]:
data = get_data_from_files()

Processing data samples
	Processing: data_A
                                             lep_pt  mcWeight
entry                                                        
0        [49138.344, 43451.65, 43018.27, 27352.213]       0.0
1       [175558.73, 138846.67, 106696.53, 79776.73]       0.0
2       [65284.066, 37290.766, 12426.926, 8804.564]       0.0
3       [74583.66, 15147.436, 10816.199, 10170.412]       0.0
4        [146784.7, 127539.97, 33562.18, 29776.129]       0.0
5        [135680.69, 82894.8, 62508.098, 42552.906]       0.0
6         [41998.89, 29384.5, 20661.203, 7902.9326]       0.0
7       [67713.18, 42791.816, 12742.542, 10291.093]       0.0
8       [133997.06, 131716.92, 121551.82, 95291.92]       0.0
9        [48983.41, 17643.121, 10876.828, 9635.456]       0.0
10         [55937.367, 29541.4, 22381.6, 7205.3184]       0.0
11     [39499.785, 24443.102, 18717.549, 11692.006]       0.0
12        [58112.008, 37625.016, 9331.23, 8709.875]       0.0
13        [38138.34, 9698.

                                             lep_pt     mcWeight
entry                                                           
0       [63834.66, 21530.459, 14459.594, 7744.2534]  1941.640015
1       [45412.223, 45201.867, 12077.552, 8478.665]  1941.109985
2        [634492.4, 45130.008, 43174.137, 11346.41]  1941.140015
3           [39319.23, 37666.0, 8385.744, 7025.795]  1942.939941
4        [34908.57, 27585.71, 10877.595, 10269.535]  1941.010010
...                                             ...          ...
893     [39341.17, 36342.305, 16145.819, 10202.409]  1941.119995
894       [33576.36, 13298.792, 9738.793, 9630.099]  1941.260010
895    [45759.707, 39300.016, 7871.5405, 7596.5938]  1939.670044
896      [42553.797, 37560.52, 9861.585, 7430.7646]  1940.939941
897     [62782.72, 34426.656, 33521.055, 17723.377]  1941.189941

[898 rows x 2 columns]
                                            lep_pt       mllll
entry                                                         
1    

                                              lep_pt  mcWeight
entry                                                         
212830    [36972.582, 32431.45, 22276.613, 8554.423]  0.100000
212831  [112417.19, 108706.86, 36159.527, 22437.623]  1.000000
212832   [68624.91, 39446.203, 27194.484, 13616.549]  0.315802
212833  [43951.297, 42755.316, 35177.926, 28164.555]  1.000000
212834   [64409.938, 23225.396, 18013.217, 8719.718]  4.512200
...                                              ...       ...
319240     [79276.97, 74099.34, 38473.66, 24260.059]  0.140936
319241       [60185.34, 54156.5, 40993.6, 15365.876]  0.124487
319242     [75172.12, 54085.28, 32118.32, 22810.438]  1.000000
319243     [25419.133, 13604.626, 9381.124, 8718.43]  1.000000
319244    [83411.27, 66432.57, 47565.496, 45587.223]  1.000000

[106415 rows x 2 columns]
                                                   lep_pt        mllll
entry                                                                 
212855      

                                              lep_pt   mcWeight
entry                                                          
104555    [46001.96, 38979.13, 29823.256, 12602.319]  28.485731
104556   [56650.215, 35651.234, 16801.18, 12042.273]  28.485731
104557  [56644.105, 21678.346, 21660.598, 13688.939] -28.485731
104558   [61411.67, 31524.512, 29036.105, 16797.668]  28.485731
104559  [43267.254, 29930.846, 13864.332, 10679.079]  28.485731
...                                              ...        ...
164711  [32143.482, 24158.068, 17203.547, 14358.152]  28.485731
164712   [39488.273, 33694.094, 32709.998, 14797.52]  28.485731
164713     [63284.21, 22707.84, 15635.994, 14873.25]  28.485731
164714    [52538.805, 40321.457, 25766.85, 19381.92]  28.485731
164715   [58730.094, 35112.83, 12265.971, 11089.536]  28.485731

[60161 rows x 2 columns]
                                                   lep_pt       mllll
entry                                                                
10

IndexError: indexes 16065856:16352787 are beyond the end of data source 'https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/4lep/MC/mc_344235.VBFH125_ZZ4lep.4lep.root'

In [ ]:
def plot_data(data):
    GeV = 1.0
    xmin = 80 * GeV
    xmax = 250 * GeV
    step_size = 5 * GeV

    bin_edges = np.arange(start=xmin, # The interval includes this value
                     stop=xmax+step_size, # The interval doesn't include this value
                     step=step_size ) # Spacing between values
    bin_centres = np.arange(start=xmin+step_size/2, # The interval includes this value
                            stop=xmax+step_size/2, # The interval doesn't include this value
                            step=step_size ) # Spacing between values

    data_x,_ = np.histogram(data['data']['mllll'], 
                            bins=bin_edges ) # histogram the data
    data_x_errors = np.sqrt( data_x ) # statistical error on the data

    signal_x = data[r'Signal ($m_H$ = 125 GeV)']['mllll'] # histogram the signal
    signal_weights = data[r'Signal ($m_H$ = 125 GeV)'].totalWeight # get the weights of the signal events
    signal_color = samples[r'Signal ($m_H$ = 125 GeV)']['color'] # get the colour for the signal bar

    mc_x = [] # define list to hold the Monte Carlo histogram entries
    mc_weights = [] # define list to hold the Monte Carlo weights
    mc_colors = [] # define list to hold the colors of the Monte Carlo bars
    mc_labels = [] # define list to hold the legend labels of the Monte Carlo bars

    for s in samples: # loop over samples
        if s not in ['data', r'Signal ($m_H$ = 125 GeV)']: # if not data nor signal
            mc_x.append( data[s]['mllll'] ) # append to the list of Monte Carlo histogram entries
            mc_weights.append( data[s].totalWeight ) # append to the list of Monte Carlo weights
            mc_colors.append( samples[s]['color'] ) # append to the list of Monte Carlo bar colors
            mc_labels.append( s ) # append to the list of Monte Carlo legend labels
    


    # *************
    # Main plot 
    # *************
    main_axes = plt.gca() # get current axes
    
    # plot the data points
    main_axes.errorbar(x=bin_centres, y=data_x, yerr=data_x_errors,
                      fmt='ko', # 'k' means black and 'o' is for circles 
                      label='Data') 
    
    # plot the Monte Carlo bars
    mc_heights = main_axes.hist(mc_x, bins=bin_edges, 
                                weights=mc_weights, stacked=True, 
                                color=mc_colors, label=mc_labels )
    
    mc_x_tot = mc_heights[0][-1] # stacked background MC y-axis value
    
    # calculate MC statistical uncertainty: sqrt(sum w^2)
    mc_x_err = np.sqrt(np.histogram(np.hstack(mc_x), bins=bin_edges, weights=np.hstack(mc_weights)**2)[0])
    
    # plot the signal bar
    main_axes.hist(signal_x, bins=bin_edges, bottom=mc_x_tot, 
                   weights=signal_weights, color=signal_color,
                   label=r'Signal ($m_H$ = 125 GeV)')
    
    # plot the statistical uncertainty
    main_axes.bar(bin_centres, # x
                  2*mc_x_err, # heights
                  alpha=0.5, # half transparency
                  bottom=mc_x_tot-mc_x_err, color='none', 
                  hatch="////", width=step_size, label='Stat. Unc.' )

    # set the x-limit of the main axes
    main_axes.set_xlim( left=xmin, right=xmax ) 
    
    # separation of x axis minor ticks
    main_axes.xaxis.set_minor_locator( AutoMinorLocator() ) 
    
    # set the axis tick parameters for the main axes
    main_axes.tick_params(which='both', # ticks on both x and y axes
                          direction='in', # Put ticks inside and outside the axes
                          top=True, # draw ticks on the top axis
                          right=True ) # draw ticks on right axis
    
    # x-axis label
    main_axes.set_xlabel(r'4-lepton invariant mass $\mathrm{m_{4l}}$ [GeV]',
                        fontsize=13, x=1, horizontalalignment='right' )
    
    # write y-axis label for main axes
    main_axes.set_ylabel('Events / '+str(step_size)+' GeV',
                         y=1, horizontalalignment='right') 
    
    # set y-axis limits for main axes
    main_axes.set_ylim( bottom=0, top=np.amax(data_x)*1.6 )
    
    # add minor ticks on y-axis for main axes
    main_axes.yaxis.set_minor_locator( AutoMinorLocator() ) 

    # Add text 'ATLAS Open Data' on plot
    plt.text(0.05, # x
             0.93, # y
             'ATLAS Open Data', # text
             transform=main_axes.transAxes, # coordinate system used is that of main_axes
             fontsize=13 ) 
    
    # Add text 'for education' on plot
    plt.text(0.05, # x
             0.88, # y
             'for education', # text
             transform=main_axes.transAxes, # coordinate system used is that of main_axes
             style='italic',
             fontsize=8 ) 
    
    # Add energy and luminosity
    lumi_used = str(lumi*fraction) # luminosity to write on the plot
    plt.text(0.05, # x
             0.82, # y
             '$\sqrt{s}$=13 TeV,$\int$L dt = '+lumi_used+' fb$^{-1}$', # text
             transform=main_axes.transAxes ) # coordinate system used is that of main_axes
    
    # Add a label for the analysis carried out
    plt.text(0.05, # x
             0.76, # y
             r'$H \rightarrow ZZ^* \rightarrow 4\ell$', # text 
             transform=main_axes.transAxes ) # coordinate system used is that of main_axes

    # draw the legend
    main_axes.legend( frameon=False ) # no box around the legend
    
    return

In [ ]:
plot_data(data)